In [ ]:
from maggy import experiment

### 1. Model definition

In [ ]:
# Model definition.
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):

    def __init__(self, l1_size, l2_size, l3_size):
        super().__init__()
        self.linear1 = nn.Linear(2,l1_size)
        self.linear2 = nn.Linear(l1_size,l2_size)
        self.linear3 = nn.Linear(l2_size,l3_size)
        self.output = nn.Linear(l3_size, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.relu(self.linear3(x))
        return self.output(x)

model = NeuralNetwork(24,24,24)

### 2. Dataset creation

In [ ]:
# Dataset generator.
x = torch.reshape(torch.rand(1000,2), (-1,2))
y = torch.reshape(x[:,0] * torch.exp(x[:,0]**2 - x[:,1]**2), (-1,1))

dataset = torch.utils.data.TensorDataset(x, y)

train_set, test_set = torch.utils.data.random_split(dataset, [800,200])

### 3. Defining the training function

In [1]:
def training_function(model, train_set, test_set):
    
    from pathlib import Path
    import os
    import random
    import numpy as np

    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader    
    
    
    def evaluate(model, test_loader):
        model.eval()
        with torch.no_grad():
            for data in test_loader:
                x, y = data[0], data[1]
                outputs = model(x)
                loss = criterion(outputs, y)
        return loss

    # Define training parameters
    num_epochs = 20
    batch_size = 256
    learning_rate = 0.1

    train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
    test_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=False)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)

    for epoch in range(num_epochs):
        print(f"Epoch: {epoch}, Training ... \n")
        if epoch % 10 == 1:
            loss = evaluate(model=model, test_loader=test_loader)
            print("-"*75)
            print(f"Epoch: {epoch}, Loss: {loss}")
            print("-"*75)

        model.train()
        for data in train_loader:
            inputs, y = data[0], data[1]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

    loss = evaluate(model=model, test_loader=test_loader)
    print("-"*75)
    print(f"Final test loss: {loss}")
    print("-"*75)
    loss = loss.item()

    return loss

### 4. Configuring the experiment 

In [ ]:
from maggy.experiment_config import DistributedConfig

config = DistributedConfig(name="torch_test", model=model, train_set=train_set, test_set=test_set)

### 5. Run distributed training

In [ ]:
experiment.lagom(training_function, config)